## Introdução

Este trabalho foi realizado por Filipe C. L. Duarte (fcld), Hélio Gonçalves de Souza Junior (hgsj) e Matheus de Farias Cavalcanti Santos (mfcs) como parte da avaliação da disciplina de Aprendizagem de Máquina da Pós-Graduação em Ciências da Computação da UFPE no primeiro semestre de 2019. 


1. [Carregando bibliotecas](#carregando_bibliotecas)
2. [Conhecendo os dados](#conhecendo_dados)
3. [Estatística descritiva](#estat_descritiva)
4. [Pré-processamento](#pre_process)
5. [Análise exploratória de dados](#eda)
7. [Modelos](#modelos)
8. [Autoencoder](#autoencoder)
9. [Modelos após redução da dimensionalidade](#modelosauto)
10. [XGBoost com Grid Search](#xgboost_grid)

<a id='carregando_bibliotecas'></a>

## Carregando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
pd.set_option('display.max_columns', 100)

## Loading data

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

<a class="anchor" id="conhecendo_dados"></a>

## Conhecendo os dados

Algumas informações sobre os dados:

* as variáveis com o sufixo **"bin"** são binárias; e **"cat"** para indicar variáveis categóricas.
* Os dados omissos **missing data** foram codificados com o valor **-1** . 
* A variável de interesse, **target** significa se o segurado incorreu em sinistou ou não. 

Portanto, o objetivo deste prejeto é o de construir um algoritmo de classificação com o objetivo de prever se o segurado irá sinistrar. 


5 primeiras linhas dos dados de treinamento

In [ ]:
train.head()

In [ ]:
train.info()

5 últimas linhas dos dados de treinamento

In [ ]:
train.tail()

A partir desses dados de treinamento, vemos:
* variáveis binárias
* variáveis categóricas codigicadas como valores inteiros
* variáveis restantes como valores reais (float) ou inteiros
* -1 representando os valores faltantes
* a variável **target** e o **id**

Visualizando a quantidade de linhas e colunas dos dados de treinamento.

In [ ]:
train.shape

Temos 59 colunas e 595212 linhas. <br>
Vamos analisar abaixo se há duplicação de linhas.



In [ ]:
# test head
test.head()

O conjunto de dados de teste também possui a variável **id** que representa a identificação do segurado. 
Vamos excluí-la de ambos os conjuntos de dados (treinamento e teste).

In [ ]:
train.drop_duplicates()
train = train.drop(['id'], axis = 1)
train.shape

Não há linhas em duplicidade.
Precisamos verificar a estrutura da tabela dos dados de teste para ver se está semelhante.

In [ ]:
test.drop_duplicates()
test_id = test['id']
test = test.drop(['id'], axis = 1)
test.shape

Percebemos que falta uma coluna no conjunto de teste. Qual seria?

In [ ]:
test.head()

Falta a variável **target**. <br>
Tudo bem, vamos analisar a estrutura dos dados com a função `info()`.

<a class="anchor" id="estat_descritiva"></a>

## Estatística descritiva

O método `describe` apresenta as estatísticas descritivas para todas as colunas do data frame. 
Contudo, só fará sentido aplicá-lo nas variáveis contínuas, isto é, naquelas representadas pelo conjunto dos valores reais. 
Para analisar as variáveis categóricas, utilizaremos os gráficos na análise exploratória. 

#### Criando vetores com os nome das variáveis pelos grupos (reg, bin, car ...)

In [ ]:
colunas = train.columns.tolist()
colunas_reg = [col for col in colunas if 'reg' in col]
colunas_cat = [col for col in colunas if 'cat' in col]
colunas_bin = [col for col in colunas if 'bin' in col]
colunas_car = [col for col in colunas if 'car' in col and 'cat' not in col]
colunas_calc = [col for col in colunas if 'calc' in col]
print(colunas_cat)

In [ ]:
train.loc[:,colunas_reg].describe()

Apenas a variável **reg_03** possui missing. <br>
Para solucionar esse problema, vamos usar um método de imputar a mediana onde existe o valor **-1**. <br>
Após, faremos a normalização **min-max**  em **reg_02** e **reg_03** para reduzir a escala, fixando-as no intervalor [0,1].

#### variáveis 'car'

In [ ]:
train.loc[:, colunas_car].describe()

Apenas ps_car_12 e ps_car_15 possuem dados faltantes
Vamos aplicar a normalização **min-max** para padronizar a escala. 

### Qual a proporção de segurados com sinistros (priori).

In [ ]:
id_0 = train[train.target == 0].index
id_1 = train[train.target == 1].index
proporção = id_1.shape[0]/train.shape[0]
print('Qual a probabilidade a priori de um segurado sinistrar: {}'. format(proporção))

<a class="anchor" id="pre_process"></a>

## Pré-processamento

### Checando os valores omissos (*missing*)
Os valores omissos foram representados pelo valor -1.

In [ ]:
var_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        var_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variável {} tem {} exemplos ({:.2%}) com valores omissos'.format(f, missings, missings_perc))
        
print('No total, existem {} variáveis com valores omissos'.format(len(var_missing)))

- **ps_car_03_cat and ps_car_05_cat** têm uma elevada quantidade de dados omissos - optamos pela remoção dessas variáveis.
- Para a variável **ps_reg_03** e **ps_car_14**, aplicaremos a imputação da média.
- Enquanto que, para **ps_car_11**, imputaremos a moda (valor mais frequente).
- Para as demais variáveis categóricas, optamos por deixar o missing como uma característica. 

In [ ]:
# Excluindo variáveis com muitos dados omissos
variaveis_excluir = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(variaveis_excluir, inplace=True, axis=1)
train.drop(colunas_calc, inplace = True, axis = 1)
# Imputando com a média e a moda
media_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
moda_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
train['ps_reg_03'] = media_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_14'] = media_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = moda_imp.fit_transform(train[['ps_car_11']]).ravel()

<a class="anchor" id="eda"></a>

## Análise exploratória de dados

### Categóricas
Vamos criar gráficos de barras para as variáveis categóricas e analisar como está a distribuição e os dados omissos.

In [ ]:
# excluir a variável 'ps_car_03_cat' e 'ps_car_05_cat'
colunas_cat.remove('ps_car_03_cat')
colunas_cat.remove('ps_car_05_cat')
print(colunas_cat)

In [ ]:
for i in colunas_cat:
    plt.figure()
    fig, ax = plt.subplots(figsize = (20,10))
    sns.barplot(ax = ax, x = i, y = 'target', data = train)
    plt.ylabel('% target', fontsize = 18)
    plt.xlabel(i, fontsize = 18)
    plt.tick_params(axis = 'both', which= 'major', labelsize = 18)
    plt.show()

Verificamos que os dados omissos (missing) são representativos. Dessa forma, optamos por deixá-los como uma categoria adicional, pois o segurado que não apresentou informação apresenta maior probabilidade de sinistrar (acidentar).

### Variáveis contínuas - Correlação

In [ ]:
continuas = [colunas_reg, colunas_car]
def correl(t):
    correlacao = train[t].corr()
    cmap = sns.diverging_palette(220, 10, as_cmap = True)

    fig, ax = plt.subplots(figsize = (10,10))
    sns.heatmap(correlacao, cmap = cmap, vmax = 1.0, center = 0, fmt = '.2f',
           square = True, linewidths = .5, annot = True, cbar_kws ={"shrink": .75})
    plt.show();
    
# Variáveis reg
for j in continuas:
    print('Heat Map de correlações para as variáveis {}' .format(j))
    correl(j)

Verificamos correlação forte para as variáveis:
- ps_reg_02 e ps_reg_03 (0,7)
- ps_car_12 e ps_car_13 (0,67)
- ps_car_12 e ps_car_14 (0,58)
- ps_car_13 and ps_car_15 (0,67)

#### Codificação das variáveis 'int64' como 'category' 
Fizemos a codificação em `category` para realizar o procedimento de one-hot-encoding das variáveis categóricas.

In [ ]:
for i in train.columns:
    if train[i].dtype == 'int64' and i != 'target':
        train[i] = train[i].astype('category')

In [ ]:
# Checando
train.info()

### Realizando one-hot-encoding nas variáveis categóricas
Para as variáveis que possuem mais de 2 categorias, realizamos o processo de one-hot-encoding que é a criação de atributos (variáveis) para cada categoria da variável. Essas variáveis serão binárias, assumindo o valor 1 quando da presença da categoria, e 0, na ausência.

In [ ]:
# função get_dummies transforma as categorias em variáveis binárias
train = pd.get_dummies(train)
train.head()

In [ ]:
# checando a dimensão
train.shape

**Criamos as variáveis binárias. 

Vamos separar a variável **target** dos atributos, criando uma matriz X e um vetor Y.

In [ ]:
X = train.drop(["target"], axis = 1)
y = train["target"]

<a class="anchor" id="modelos"></a>

## Modelos

#### Divindindo os dados em treinamento e teste
Vamos dividir os dados em treinamento e teste para realizar a avaliação por meio da validação cruzada com o intuito de melhorar a capacidade de generalização dos classificadores. 
Usamos a função `train_test_split` do sklearn com a amostragem estratificada para manter a proporção da variável target. 

#### Normalização min-max 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=0)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

### Coeficiente de Gini Normalizado

Utilizamos a implementação do coeficiente de gini normalizado em python obtido neste tópico: http://www.kaggle.com/c/ClaimPredictionChallenge/discussion/703

In [ ]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

### Regressão logística

Primeiro teste da regressão logística com a penalização L2. 
Usamos o pipeline para fazer a normalização min-max.

In [ ]:
# Testando com a regressão logística com penalização L2
lr = LogisticRegression(penalty='l2', random_state=1)
lr.fit(X_train, y_train)
prob = lr.predict_proba(X_test)[:,1]
print("Índice de Gini normalizado para a Regressão Logística: ",gini_normalized(y_test, prob))

### Random Forest

Testamos também o Random Forest com 200 árvores, 15 atributos e com profundidade máxima de 4 nós.

In [ ]:
# Random Forest com 20 árvores
rf = RandomForestClassifier(n_estimators = 20, max_depth = 4, random_state = 1, max_features = 20)
rf.fit(X_train, y_train)
predictions_prob = rf.predict_proba(X_test)[:,1]
print("Índice de Gini normalizado para o Random Forest: ", gini_normalized(y_test, predictions_prob))

### XGBoost 

Testamos o XGBoost com 100 estimações, profundidade máxima de 5, e taxa de aprendizagem de 0.05.

In [ ]:
# taxa de aprendizagem = 0.05
xgbm = XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.05, random_state = 1)
xgbm.fit(X_train, y_train)
prob_xgb = xgbm.predict_proba(X_test)[:,1]
print("--------------------------------------------------------------------------------------------")
print("Índice de Gini normalizado para o XGBoost com learning_rate = 0.05: ", gini_normalized(y_test, prob_xgb))
print("--------------------------------------------------------------------------------------------")

In [ ]:
# Optamos por tentar submeter o modelo com melhor score nos dados X_test - XGBoost com . 
# Testaremos nos dados completo de treinamento.
prob_xgb_y = xgbm.predict_proba(X)[:,1]
print("--------------------------------------------------------------------------------------------")
print("Índice de Gini normalizado para o XGBoost com learning_rate = 0.05 dados treino: ", gini_normalized(y, prob_xgb_y))
print("--------------------------------------------------------------------------------------------")

Aplicar esses procedimentos aos dados de teste

In [ ]:
# Excluindo variáveis com muitos dados omissos
variaveis_excluir = ['ps_car_03_cat', 'ps_car_05_cat']
test.drop(variaveis_excluir, inplace=True, axis=1)
test.drop(colunas_calc, inplace = True, axis = 1)
# Imputando com a média e a moda
media_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
moda_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
test['ps_reg_03'] = media_imp.fit_transform(test[['ps_reg_03']]).ravel()
test['ps_car_14'] = media_imp.fit_transform(test[['ps_car_14']]).ravel()
test['ps_car_11'] = moda_imp.fit_transform(test[['ps_car_11']]).ravel()

# categorizando 
for i in test.columns:
    if test[i].dtype == 'int64' and i != 'target':
        test[i] = test[i].astype('category')
        
# função get_dummies transforma as categorias em variáveis binárias
test = pd.get_dummies(test)

In [ ]:
# Normalização
scaler = MinMaxScaler()
scaler.fit(test)
test = scaler.transform(test)

In [ ]:
# Aplicando no modelo XGBoost
y_test = xgbm.predict_proba(test)[:,1]

# Em results_df está a base de teste escorada, a coluna target possui as probabilidades
results_df = pd.DataFrame(data={'id':test_id, 'target':y_test})
print(results_df)
results_df.to_csv('submissão_1.csv', index=False)

Verificamos que os modelos não produziram bons resultados em comparação com o score da competição.
Portanto, vamos tentar os modelos modelos com os dados sem pré-processamento, apenas com a normalização min-max. 
Optamos por essa abordagem em razão de que os competidores que alcançaram bons resultados não excluíram variáveis, e não realizaram a exclusão de dados missing. 

### Testando com dados brutos

In [ ]:
# Testando nos dados sem exclusão de variáveis
#train = pd.read_csv('../input/train.csv')
#X = train.iloc[:,2:]
#y = train.iloc[:,1:2]
#X = train.drop(["id","target"], axis = 1)
#y = train["target"]
# Separando um conjunto para avaliar
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, stratify=y, random_state=0)
# Normalização Min-Max
#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#scaler = MinMaxScaler()
#scaler.fit(X_test)
#X_test = scaler.transform(X_test)
# Modelos
# Regressão Logística
# Testando com a regressão logística com penalização L2
#lr = LogisticRegression(penalty='l2', random_state=1)
#lr.fit(X_train, y_train)
#prob_lr = lr.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para a Regressão Logística: ",gini_normalized(y_test, prob_lr))
#print("--------------------------------------------------------------------------------------------")
# Random Forest com 200 árvores
#rf = RandomForestClassifier(n_estimators = 200, max_depth = 4, random_state = 1, max_features = 15)
#rf.fit(X_train, y_train)
#prob_rf = rf.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o Random Forest: ", gini_normalized(y_test, prob_rf))
#print("--------------------------------------------------------------------------------------------")
# XGBoost 
#xgbm = XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.05, random_state = 1)
#xgbm.fit(X_train, y_train)
#prob_xgb = xgbm.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o XGBoost: ", gini_normalized(y_test, prob_xgb))
#print("--------------------------------------------------------------------------------------------")
# LightGBM
#lgb = LGBMClassifier(n_estimators = 100, learning_rate = 0.02, subsample = 0.7, num_leaves = 15, seed = 1)
#lgb.fit(X_train, y_train)
#prob_lgb = lgb.predict_proba(X_test)[:, 1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o LightGBM: ", gini_normalized(y_test, prob_lgb))
#print("--------------------------------------------------------------------------------------------")



Percebe-se claramente que os modelos apresentaram maiores valores para o coeficiente de gini normalizado, isto é, o tratamento dos dados (exclusão de variáveis e imputação de média e moda) não ajudou. 
Dessa forma, tentaremos rodar um autoencoder que serve para reduzir a dimensionalidade dos atributos. 

<a class="anchor" id="autoencoder"></a>

# Autoencoder

#### Importação das bibliotecas necessárias

In [ ]:
import pandas as pd
from pandas import read_csv, DataFrame
import numpy as np
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets
from keras.layers import Input, Dense
from keras.models import Model
from matplotlib import pyplot as plt

#### Carregamento e normalização da base de treinamento

In [ ]:
# Carregamento das bases de treinamento e teste em dataframes
train = pd.read_csv('../input/train.csv')

print(train.shape)

# X armazena dos dados em um dataframe
X = train.iloc[:,2:]
# y armazena os labels em um dataframe
y = train.iloc[:,1:2]

# target_names armazena os valores distintos dos labels
target_names = train['target'].unique()

# Normaliza os dados de treinamento
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

print(X_scaled)

In [ ]:
X.head()

In [ ]:
X.columns

In [ ]:
print("Número de Colunas: ", X.shape[1])

In [ ]:
y.head()

#### Configuração e execução do treinamento do Autoencoder

In [ ]:
# Criação do AutoEncoder com 3 neurônios na camada escondida usando Keras.
#input_dim = X_scaled.shape[1]

# Definição do número de variáveis resultantes do Encoder
#encoding_dim = 10

#input_data = Input(shape=(input_dim,))

# Configurações do Encoder
#encoded = Dense(encoding_dim, activation='linear')(input_data)
#encoded = Dense(encoding_dim, activation='sgmoid')(input_data)
#encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_data)

#encoded1 = Dense(20, activation = 'relu')(input_data)
#encoded2 = Dense(10, activation = 'relu')(encoded1)
#encoded3 = Dense(5, activation = 'relu')(encoded2)
#encoded4 = Dense(encoding_dim, activation = 'relu')(encoded3)

# Configurações do Decoder
#decoded = Dense(input_dim, activation='linear')(encoded)
#decoded = Dense(input_dim, activation='sgmoid')(encoded)

#decoded1 = Dense(5, activation = 'relu')(encoded4)
#decoded2 = Dense(10, activation = 'relu')(decoded1)
#decoded3 = Dense(20, activation = 'relu')(decoded2)
#decoded4 = Dense(input_dim, activation = 'sigmoid')(decoded3)

# Combinando o Encoder e o Decoder em um modelo AutoEncoder
#autoencoder = Model(input_data, decoded4)
#autoencoder.compile(optimizer='adam', loss='mse')
#print(autoencoder.summary())
# Treinamento de fato - Definição de alguns parâmetros como número de épocas, batch size, por exemplo.
#history = autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=256, shuffle=True, validation_split=0.1, verbose = 1)

#plot our loss 
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('Model Train vs Validation Loss')
#plt.ylabel('Loss')
#plt.xlabel('Epoch')
#plt.legend(['Train', 'Validation'], loc='upper right')
#plt.show()

#### Utilização do Encoder gerado para realizar a compressão e reduzir a dimensão da base de treinamento

In [ ]:
#test = pd.read_csv('../input/test.csv')

#print(test.shape)

# X armazena dos dados em um dataframe
#X = test.iloc[:,1:]

# Normaliza os dados de treinamento
#scaler = MinMaxScaler()
#scaler.fit(X)
#X_scaled = scaler.transform(X)

# Utilizar o Encoder para codificar os dados de entrada
#encoder = Model(input_data, encoded4)
#encoded_data = encoder.predict(X_scaled)

#print(encoded_data)

<a class="anchor" id="modelosauto"></a>

## Avaliação dos Modelos após o Autoencoder

In [ ]:
# Carregamento das bases de treinamento e teste em dataframes
#train = pd.read_csv('../input/train.csv')
#X = train.drop(["id","target"], axis = 1)
#y = train["target"]

# Separando um conjunto para avaliar
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

# Normalização Min-Max
#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#scaler = MinMaxScaler()
#scaler.fit(X_test)
#X_test = scaler.transform(X_test)

# aplicando autoencoder nos dados de treinamento e teste
#encoder = Model(input_data, encoded4)
#encoded_data_train = encoder.predict(X_train)
#encoder = Model(input_data, encoded4)
#encoded_data_test = encoder.predict(X_test)

# Modelos
# Regressão Logística
#lr = LogisticRegression(penalty='l2', random_state=1)
#lr.fit(encoded_data_train, y_train)
#prob_lr = lr.predict_proba(encoded_data_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para a Regressão Logística com autoencoder: ",gini_normalized(y_test, prob_lr))
#print("--------------------------------------------------------------------------------------------")


# Random Forest com 200 árvores
#rf = RandomForestClassifier(n_estimators = 200, max_depth = 5, random_state = 1, max_features = 7)
#rf.fit(encoded_data_train, y_train)
#prob_rf = rf.predict_proba(encoded_data_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o Random Forest com autoencoder: ", gini_normalized(y_test, prob_rf))
#print("--------------------------------------------------------------------------------------------")


# XGBoost 
#import xgboost as xgb
#xgbm = xgb.XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.05, random_state = 1)
#xgbm.fit(encoded_data_train, y_train)
#prob_xgb = xgbm.predict_proba(encoded_data_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o XGBoost com autoencoder: ", gini_normalized(y_test, prob_xgb))
#print("--------------------------------------------------------------------------------------------")


# LightGBM
#from lightgbm import LGBMClassifier
#lgb = LGBMClassifier(n_estimators = 100, learning_rate = 0.02, subsample = 0.7, num_leaves = 15, seed = 1)
#lgb.fit(encoded_data_train, y_train)
#prob_lgb = lgb.predict_proba(encoded_data_test)[:, 1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o LightGBM com autoencoder: ", gini_normalized(y_test, prob_lgb))
#print("--------------------------------------------------------------------------------------------")

<a class="anchor" id="xgboost_grid"></a>
# XGBoost com Grid Search

In [ ]:
#import warnings
#warnings.filterwarnings('ignore')
#import numpy as np
#import pandas as pd
#from datetime import datetime
#from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
#from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import train_test_split
#from xgboost import XGBClassifier

In [ ]:
#def timer(start_time=None):
#    if not start_time:
#        start_time = datetime.now()
#        return start_time
#    elif start_time:
#        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
#        tmin, tsec = divmod(temp_sec, 60)
#        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


# Carregamento das bases de treinamento e teste em dataframes
#train = pd.read_csv('../input/train.csv', dtype={'id': np.int32, 'target': np.int8})
#X = train.drop(["id","target"], axis = 1)
#Y = train["target"].values

# Separando um conjunto para avaliar
#X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, random_state=0)

######################################

#train_df = pd.read_csv('../input/train.csv', dtype={'id': np.int32, 'target': np.int8})
#Y = train_df['target'].values
#X = train_df.drop(['target', 'id'], axis=1)
#test_df = pd.read_csv('../input/test.csv', dtype={'id': np.int32})
#test = test_df.drop(['id'], axis=1)

In [ ]:
# Grid de Parâmetros para o XGBoost
#params = {
    #    'min_child_weight': [1, 5, 10],
   #     'gamma': [0.5, 1, 1.5, 2, 5],
  #      'subsample': [0.6, 0.8, 1.0],
 #       'colsample_bytree': [0.6, 0.8, 1.0],
#        'max_depth': [3, 4, 5]}

In [ ]:
#xgb = XGBClassifier(learning_rate=0.02, n_estimators=100, objective='binary:logistic', silent=True, nthread=1)

In [ ]:
# Número de folds para Cross-Validation
#folds = 2

# Número de combinações a serem feitos no Grid Search. No Total podem ser feitas 3x5x3x3x3 = 405 combinações. Quantos mais combinações, mais tempo leva.
#param_comb = 1

# Configuração dos folds estratificados para o Cross-Validation
#skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# Configuração do Grid Search
#random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )
#grid = GridSearchCV(estimator=xgb, param_grid=params ,scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3)

# Execução do Treinamento com Grid Search
#start_time = timer(None) # timing starts from this point for "start_time" variable
#random_search.fit(X, Y)
#grid.fit(X, Y)
#timer(start_time) # timing ends here for "start_time" variable

In [ ]:
#print('\n All results:')
#print(random_search.cv_results_)
#print('\n Best estimator:')
#print(random_search.best_estimator_)
#print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
#print(random_search.best_score_ * 2 - 1)
#print('\n Best hyperparameters:')
#print(random_search.best_params_)
#results = pd.DataFrame(random_search.cv_results_)
#results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

# print('\n All results:')
# print(grid.cv_results_)
# print('\n Best estimator:')
# print(grid.best_estimator_)
# print('\n Best score:')
# print(grid.best_score_ * 2 - 1)
# print('\n Best parameters:')
# print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)
# results.to_csv('xgb-grid-search-results-01.csv', index=False)

# y_test = grid.best_estimator_.predict_proba(test)
# results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
# results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

In [ ]:
# predict_proba já utiliza o modelo com os melhores hyperparâmetros para realizar o predict da base de teste.
#y_test = random_search.predict_proba(test)

# Em results_df está a base de teste escorada, a coluna target possui as probabilidades
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#print(results_df)
#results_df.to_csv('submission-random-grid-search-xgb-porto-01.csv', index=False)

## TESTE - FINAL

In [ ]:
# Carregando Bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
pd.set_option('display.max_columns', 100)



In [ ]:
# Computing gini coefficient ( Coursey Kaggle)
# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation @jit
#def eval_gini(y_true, y_prob):
#    y_true = np.asarray(y_true)
#    y_true = y_true[np.argsort(y_prob)]
#    ntrue = 0
#    gini = 0
#    delta = 0
#    n = len(y_true)
#    for i in range(n-1, -1, -1):
#        y_i = y_true[i]
#        ntrue += y_i
#        gini += y_i * delta
#        delta += 1 - y_i
#    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
#    return gini

In [ ]:
# IMPORTAÇÃO DOS DADOS
#train = pd.read_csv('../input/train.csv')
#test = pd.read_csv('../input/test.csv')

In [ ]:
# Separando X e y
#X = train.drop(["id","target"], axis = 1)
#y = train["target"].values

In [ ]:
# Excluindo variáveis com 'calc'
#colunas = X.columns.tolist()
#colunas_cat = [col for col in colunas if 'cat' in col]
#colunas_calc = [col for col in colunas if 'calc' in col]

#variaveis_excluir = colunas_calc
#X.drop(variaveis_excluir, inplace=True, axis=1)

In [ ]:
# One-hot nas categóricas
#X = pd.get_dummies(X)
#X.head()

In [ ]:
# Separando um conjunto para avaliar
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, stratify=y, random_state=0)
# Normalização Min-Max
#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#scaler = MinMaxScaler()
#scaler.fit(X_test)
#X_test = scaler.transform(X_test)

In [ ]:
# Testar os modelos
# Modelos
# Regressão Logística
# Testando com a regressão logística com penalização L2
#lr = LogisticRegression(penalty='l2', random_state=1)
#lr.fit(X_train, y_train)
#prob_lr = lr.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para a Regressão Logística: ",eval_gini(y_test, prob_lr))
#print("--------------------------------------------------------------------------------------------")
# Random Forest com 200 árvores
#rf = RandomForestClassifier(n_estimators = 200, max_depth = 4, random_state = 1, max_features = 15)
#rf.fit(X_train, y_train)
#prob_rf = rf.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o Random Forest: ", eval_gini(y_test, prob_rf))
#print("--------------------------------------------------------------------------------------------")
# XGBoost 
#xgbm = XGBClassifier(max_depth=4, n_estimators=100, learning_rate=0.05, random_state = 1)
#xgbm.fit(X_train, y_train)
#prob_xgb = xgbm.predict_proba(X_test)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o XGBoost: ", eval_gini(y_test, prob_xgb))
#print("--------------------------------------------------------------------------------------------")
# LightGBM
#lgb = LGBMClassifier(n_estimators = 100, learning_rate = 0.02, subsample = 0.7, num_leaves = 15, seed = 1)
#lgb.fit(X_train, y_train)
#prob_lgb = lgb.predict_proba(X_test)[:, 1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o LightGBM: ", eval_gini(y_test, prob_lgb))
#print("--------------------------------------------------------------------------------------------")

In [ ]:
## Testando na base de treinamento completa
# Normalização Min-Max
#scaler = MinMaxScaler()
#scaler.fit(X)
#X = scaler.transform(X)

In [ ]:
# base completa
# Modelos
#prob_lr = lr.predict_proba(X)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para a Regressão Logística: ",eval_gini(y, prob_lr))
#print("--------------------------------------------------------------------------------------------")
#prob_rf = rf.predict_proba(X)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o Random Forest: ", eval_gini(y, prob_rf))
#print("--------------------------------------------------------------------------------------------")
#prob_xgb = xgbm.predict_proba(X)[:,1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o XGBoost: ", eval_gini(y, prob_xgb))
#print("--------------------------------------------------------------------------------------------")
#prob_lgb = lgb.predict_proba(X)[:, 1]
#print("--------------------------------------------------------------------------------------------")
#print("Índice de Gini normalizado para o LightGBM: ", eval_gini(y, prob_lgb))
#print("--------------------------------------------------------------------------------------------")

In [ ]:
# Criando o arquivo para submissão
#test_df = pd.read_csv('../input/test.csv', dtype={'id': np.int32})
#test = test_df.drop(['id'], axis=1)
#test.drop(variaveis_excluir, inplace=True, axis=1)
#test = pd.get_dummies(test)
#scaler = MinMaxScaler()
#scaler.fit(test)
#test = scaler.transform(test)
#y_test = xgbm.predict_proba(test)[:,1]

# Em results_df está a base de teste escorada, a coluna target possui as probabilidades
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test})
#print(results_df)
#results_df.to_csv('submission-random-grid-search-xgb-01.csv', index=False)